In [ ]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import os
import glob
import ta  # IMPORTANTE: garantir que está importado!
import warnings
warnings.filterwarnings('ignore')

DATA_DIR = "C://Users//thiag//OneDrive//ARQUIVOS//Bolsa//COLETA//MT//FUTUROS"

TIMEFRAMES = {
    'M5': mt5.TIMEFRAME_M5,
    'M15': mt5.TIMEFRAME_M15,
    'H1': mt5.TIMEFRAME_H1,
    'M2': mt5.TIMEFRAME_M2,
    'D1': mt5.TIMEFRAME_D1,    # Diário
    'W1': mt5.TIMEFRAME_W1,    # Semanal
    'MN1': mt5.TIMEFRAME_MN1   # Mensal
}

def adicionar_colunas_avancadas(df):
        for col in ['ÚLT. PREÇO', 'PREÇO ABERT.', 'PREÇO MÍN.', 'PREÇO MÁX.', 'VOL.']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
        # log_return
        df['log_return'] = np.log(df['ÚLT. PREÇO'] / df['ÚLT. PREÇO'].shift(1))
        # vol_ratio_10 e vol_ratio_20
        df['vol_ratio_10'] = df['VOL.'] / df['VOL.'].rolling(10).mean()
        df['vol_ratio_20'] = df['VOL.'] / df['VOL.'].rolling(20).mean()
        # RSI 14 e RSI 21
        df['rsi_14'] = ta.momentum.RSIIndicator(df['ÚLT. PREÇO'], window=14).rsi()
        df['rsi_21'] = ta.momentum.RSIIndicator(df['ÚLT. PREÇO'], window=21).rsi()
        # Volatility (desvio padrão do preço de fechamento)
        df['volatility_10'] = df['ÚLT. PREÇO'].rolling(window=10).std()
        df['volatility_20'] = df['ÚLT. PREÇO'].rolling(window=20).std()
        # Momentum 5 e 10
        df['momentum_5'] = ta.momentum.roc(df['ÚLT. PREÇO'], window=5)
        df['momentum_10'] = ta.momentum.roc(df['ÚLT. PREÇO'], window=10)
        # SMA 13 e EMA 21
        df['sma_13'] = df['ÚLT. PREÇO'].rolling(window=13).mean()
        df['ema_21'] = df['ÚLT. PREÇO'].ewm(span=21, adjust=False).mean()
        # price_vs_sma_13 e price_vs_ema_21
        df['price_vs_sma_13'] = (df['ÚLT. PREÇO'] - df['sma_13']) / df['sma_13']
        df['price_vs_ema_21'] = (df['ÚLT. PREÇO'] - df['ema_21']) / df['ema_21']
        # Bandas de Bollinger 20 períodos
        bb = ta.volatility.BollingerBands(df['ÚLT. PREÇO'], window=20)
        df['bb_position_20'] = (df['ÚLT. PREÇO'] - bb.bollinger_lband()) / (bb.bollinger_hband() - bb.bollinger_lband())
        # high_low_ratio
        df['high_low_ratio'] = df['PREÇO MÁX.'] / df['PREÇO MÍN.']
        # open_close_ratio
        df['open_close_ratio'] = df['PREÇO ABERT.'] / df['ÚLT. PREÇO']
        # body_ratio
        df['body_ratio'] = (df['ÚLT. PREÇO'] - df['PREÇO ABERT.']) / (df['PREÇO MÁX.'] - df['PREÇO MÍN.'])
        # price_position
        df['price_position'] = (df['ÚLT. PREÇO'] - df['PREÇO MÍN.']) / (df['PREÇO MÁX.'] - df['PREÇO MÍN.'])
        # correlation_lag_1
        df['correlation_lag_1'] = df['ÚLT. PREÇO'].rolling(window=2).corr(df['ÚLT. PREÇO'].shift(1))
        # Remover colunas auxiliares se quiser
        df.drop(['sma_13', 'ema_21'], axis=1, inplace=True)
        
        return df

class MTDataCollector:
    def __init__(self):
        if not mt5.initialize():
            print("MetaTrader5 não iniciou, erro:", mt5.last_error())
            raise Exception("Falha na inicialização do MetaTrader5")
        
        os.makedirs(DATA_DIR, exist_ok=True)
        self.existing_files = self.get_existing_files()
    
    def __del__(self):
        mt5.shutdown()

    def get_existing_files(self):
        """Mapeia arquivos existentes"""
        files = glob.glob(os.path.join(DATA_DIR, 'WDO*.csv'))
        return {os.path.basename(f) for f in files}

    def get_timeframe_data(self, symbol, timeframe, date):
        """Coleta dados de um timeframe específico para uma data"""
        start_time = datetime(date.year, date.month, date.day, 6, 0)
        end_time = datetime(date.year, date.month, date.day, 21, 00)
        
        try:
            rates = mt5.copy_rates_range(symbol, timeframe, start_time, end_time)
            if rates is None or len(rates) == 0:
                return None
                
            df = pd.DataFrame(rates)
            df['time'] = pd.to_datetime(df['time'], unit='s')
            return df
            
        except Exception as e:
            print(f"Erro ao coletar {symbol} em {timeframe} para {date.date()}: {str(e)}")
            return None

    def process_data(self, df):
        """Processa e adiciona indicadores aos dados"""
        if df is None or len(df) == 0:
            return None
            
        df = df.copy()
        
        # Cálculos básicos
        df['range'] = df['high'] - df['low']
        df['body'] = abs(df['close'] - df['open'])
        df['upper_wick'] = df['high'] - df[['open', 'close']].max(axis=1)
        df['lower_wick'] = df[['open', 'close']].min(axis=1) - df['low']
        
        # Volume
        df['volume_ema'] = df['tick_volume'].ewm(span=20).mean()
        df['volume_ratio'] = df['tick_volume'] / df['volume_ema']
        
        # Momentum e tendência
        df['price_change'] = df['close'].pct_change()
        df['price_change_pct'] = df['price_change'] * 100
        
        for period in [8, 20, 50]:
            df[f'ma_{period}'] = df['close'].rolling(window=period).mean()
        
        df['volatility'] = df['range'].rolling(window=20).std()
        df['atr'] = df['range'].rolling(window=14).mean()
        
        df['trend'] = np.where(df['ma_20'] > df['ma_50'], 1, 
                             np.where(df['ma_20'] < df['ma_50'], -1, 0))
                
        return df

    def get_filename(self, symbol, timeframe, date):
        """Gera nome do arquivo padronizado"""
        tf_name = {v: k for k, v in TIMEFRAMES.items()}[timeframe]
        return f"{symbol.replace('$', '')}_{tf_name}_{date.strftime('%Y%m%d')}.csv"

   
    def save_data(self, df, symbol, timeframe, date):
        """Salva dados com nome padronizado"""
        if df is None:
            return None
            
        filename = self.get_filename(symbol, timeframe, date)
        filepath = os.path.join(DATA_DIR, filename)
        
        # Renomear colunas para padrão B3
        column_mapping = {
            'open': 'PREÇO ABERT.',
            'high': 'PREÇO MÁX.',
            'low': 'PREÇO MÍN.',
            'close': 'ÚLT. PREÇO',
            'tick_volume': 'VOL.',
            'time': 'DATA'
        }
        
        df_save = df.rename(columns=column_mapping)
        df_save['AJUSTE'] = df_save['ÚLT. PREÇO']
        df_save['VENCTO'] = symbol.split('$')[0]
        df_save['TIMEFRAME'] = {v: k for k, v in TIMEFRAMES.items()}[timeframe]
        
        primary_columns = ['VENCTO', 'DATA', 'TIMEFRAME', 'PREÇO ABERT.', 'PREÇO MÍN.', 
                          'PREÇO MÁX.', 'ÚLT. PREÇO', 'AJUSTE', 'VOL.']
        other_columns = [col for col in df_save.columns if col not in primary_columns]
        df_save = df_save[primary_columns + other_columns]

        df_save = adicionar_colunas_avancadas(df_save)
        
        df_save.to_csv(filepath, index=False)
        return filepath

    def collect_historical_data(self, days=30):
        """Coleta dados históricos verificando arquivos existentes"""
        symbols = ['WDO$N']
        collected_files = {}
        
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        current_date = start_date
        
        print(f"\nIniciando coleta de {days} dias: {start_date.date()} até {end_date.date()}")
        print("="*70)
        
        while current_date <= end_date:
            print(f"\nProcessando data: {current_date.date()}")
            
            for symbol in symbols:
                for tf_name, timeframe in TIMEFRAMES.items():
                    filename = self.get_filename(symbol, timeframe, current_date)
                    
                    # Verificar se arquivo já existe
                    if filename in self.existing_files:
                        print(f"  {symbol} {tf_name}: Arquivo já existe")
                        continue
                    
                    try:
                        print(f"  Coletando {symbol} {tf_name}...", end=' ')
                        
                        df = self.get_timeframe_data(symbol, timeframe, current_date)
                        if df is None:
                            print("Sem dados")
                            continue
                            
                        df_processed = self.process_data(df)
                        if df_processed is None:
                            print("Erro no processamento")
                            continue
                            
                        filepath = self.save_data(df_processed, symbol, timeframe, current_date)
                        if filepath:
                            key = f"{symbol}_{tf_name}_{current_date.date()}"
                            collected_files[key] = {
                                'path': filepath,
                                'records': len(df_processed)
                            }
                            print(f"OK ({len(df_processed)} registros)")
                            self.existing_files.add(filename)
                        
                    except Exception as e:
                        print(f"Erro: {str(e)}")
                        continue
            
            current_date += timedelta(days=1)
        
        return collected_files

def print_collection_summary(collected_files):
    """Imprime resumo da coleta"""
    print("\n" + "="*70)
    print("RESUMO DA COLETA DE DADOS")
    print("="*70)
    
    if not collected_files:
        print("\nNenhum novo arquivo coletado - Todos os dados já existem")
        return
    
    for key, info in collected_files.items():
        symbol, tf, date = key.split('_')
        filepath = info['path']
        filename = os.path.basename(filepath)
        filesize = os.path.getsize(filepath) / 1024  # KB
        
        print(f"\n{symbol} - {tf} - {date}:")
        print(f"  Arquivo: {filename}")
        print(f"  Tamanho: {filesize:.1f} KB")
        print(f"  Registros: {info['records']}")

if __name__ == "__main__":
    try:
        collector = MTDataCollector()
        collected_files = collector.collect_historical_data(days=7)
        print_collection_summary(collected_files)
        
        print("\nProcesso finalizado com sucesso!")
        
    except Exception as e:
        print(f"\nErro durante a coleta: {str(e)}")
        print("Verifique se o MetaTrader5 está aberto e configurado corretamente.")


Iniciando coleta de 7 dias: 2025-05-19 até 2025-05-26

Processando data: 2025-05-19
  WDO$N M5: Arquivo já existe
  WDO$N M15: Arquivo já existe
  WDO$N H1: Arquivo já existe
  WDO$N M2: Arquivo já existe
  WDO$N D1: Arquivo já existe
  Coletando WDO$N W1... Sem dados
  Coletando WDO$N MN1... Sem dados

Processando data: 2025-05-20
  WDO$N M5: Arquivo já existe
  WDO$N M15: Arquivo já existe
  WDO$N H1: Arquivo já existe
  WDO$N M2: Arquivo já existe
  WDO$N D1: Arquivo já existe
  Coletando WDO$N W1... Sem dados
  Coletando WDO$N MN1... Sem dados

Processando data: 2025-05-21
  WDO$N M5: Arquivo já existe
  WDO$N M15: Arquivo já existe
  WDO$N H1: Arquivo já existe
  WDO$N M2: Arquivo já existe
  WDO$N D1: Arquivo já existe
  Coletando WDO$N W1... Sem dados
  Coletando WDO$N MN1... Sem dados

Processando data: 2025-05-22
  WDO$N M5: Arquivo já existe
  WDO$N M15: Arquivo já existe
  WDO$N H1: Arquivo já existe
  WDO$N M2: Arquivo já existe
  WDO$N D1: Arquivo já existe
  Coletando WD